In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output

import os
import base64
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.json_util import dumps


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read_all({}))





#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),style={'height':'10%', 'width':'10%'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H1('Justin Brown'))),
    html.Hr(),
    html.Div(
         className='row',
        style={'display': 'flex'},
        children=[
                html.Button(id='b1',n_clicks=0, children= 'Water Rescue'),
                html.Button(id='b2',n_clicks=0, children= 'Mountain or Wilderness Rescue'),
                html.Button(id='b3',n_clicks=0, children='Disaster Rescue or Individual Tracking'),
                html.Button(id='b4', n_clicks=0, children='reset')          
        ]
    ),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        sort_action = "native",
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
         page_size=100,
        style_table={'height':'300px','overflowY':'auto','overflowX':'auto'},
        style_header={
            'backgroundColor':'rgb(160,171,216)',
            'fontWeight':'heavy'        
        },
        style_data={
            'whiteSpace':'normal',
            'backgroundColor':'rgb(255,102,255)',
            
            'height':'auto'
        },
       
    
        
    ),
    html.Br(),
     html.Hr(),
     html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
# def update_styles(selected_columns):
#     return [{
#         'if': { 'column_id': i },
#         'background_color': '#D2F3FF'
#     } for i in selected_columns]

@app.callback(Output('datatable-id',"data"), 
              [Input('b1', 'n_clicks'),Input('b2', 'n_clicks'), Input('b3', 'n_clicks')
              ])
def update_dashboard(bt1,bt2,bt3):
    
    # start case
#     if (int(bt1) == 0 and int(bt2) == 0 ):
#         df = pd.DataFrame.from_records(shelter.read_all({}))
    # use higher number of button clicks to determine filter type, can you think of a better way ? ....
    if (int(bt1) >= 1):
        df = pd.DataFrame(list(shelter.read_all({ 
            "breed": {
                "$in": [
                    "Chesapeake Bay Retriever",
                    "Labrador Retriever Mix",
                    "Newfoundland"
                ]
            },
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {
                "$gte": 26.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 156.0
                    }
                }
            ]
        }
        )))
            
        
        
    elif (int(bt2)>= 1):
        df = pd.DataFrame(list(shelter.read_all({ 
            "breed": {
                "$in": [
                    "German Shepherd",
                    "Alaskan Malamute",
                    "Old English Sheepdog",
                    "Siberian Husky",
                     "Rottweiler"
                ]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {
                "$gte": 26.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 156.0
                    }
                }
            ]
        }
        )))
        
    elif (int(bt3)>= 1):
        df = pd.DataFrame(list(shelter.read_all({ 
            "breed": {
                "$in": [
                    "Doberman Pinscher",
                    "German Shepherd",
                    "Golden Retriever",
                    "Bloodhound",
                    "Rottweiler"
                ]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {
                "$gte": 20.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 300.0
                    }
                }
            ]
        }
        )))
        
    else:
        df = pd.DataFrame.from_records(shelter.read_all({}))
        
    return df.to_dict('records')

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) 
    df = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(df, names = 'breed',)
            
        ) 
        
    ]
     
    
    
    
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]

app